<a href="https://colab.research.google.com/github/PoiZeX/CloudCourse/blob/main/ex6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install firebase



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [11]:
from ipywidgets.widgets.widget_button import ButtonStyle
#@title
import time
from ipywidgets.widgets.widget_style import Style
from zmq.constants import NULL
from firebase import firebase
import IPython
from google.colab import output
import random
import ipywidgets as widgets
from ipywidgets import VBox, HBox, HTML, Layout, Box, Label
import matplotlib.pyplot as plt
import math
import re


################################################################################
#################################### CLASSES ###################################
################################################################################
class Question():
  answers = []
  def __init__(self, question, lst_options, correct):
    self.question = question
    self.answers = lst_options
    self.correct = correct
  
  def isCorrect(self, str_option):
    correctAns = self.answers[int(self.correct)-1]
    return str_option.strip() == correctAns.strip()
  
  def get_random_answers(self):
      return random.sample(self.answers, len(self.answers))

####################################################################################
class QuestionsDB():

  lst_questions = []

  def __init__(self, lst_questions):
    self.lst_questions = lst_questions

  def get_random_question(self):
    global randomQuestionAndAnswer, answers, question, radioButtonLst
    randomQuestionAndAnswer = NULL # we want to generate new item...
    if self.lst_questions:
            random_question = random.sample(self.lst_questions, 1)[0]
            self.lst_questions.remove(random_question)
            return random_question
    else:
          return
   
  def add_question_to_DB(self):
    pass

  def remove_question_from_DB(self):
    pass

#################################################################################### 
class User():
  def __init__(self, username, password, role):
    self.username = username
    self.password = password
    self.role = role
  
  def isAdmin(self):
    return self.role == "Admin"
  
  def getName(self):
    return self.username



################################################################################
############################### GLOBAL VARIABLES ###############################
################################################################################
questionsFromDB = []
btn_go_back = NULL
current_user = NULL

# Points
points = 0
quantity_of_questions=0
quantity_of_correct_questions=0

list_of_stats = []  # will be filled with number of attempts for EACH question

currentSelectedAns = ""
finalSelectedBtn = widgets.Button(description="") # null object
question = ""


randomQuestionAndAnswer = NULL
answers = []
buttonsLst = []
questionsHistory = set() # saving the 'old' questions for non repeating. Can also be done by removing those question from all DB list (but i think it's not preferred)
FIREBASE_PATH = 'https://ex6game-default-rtdb.europe-west1.firebasedatabase.app/'
style = {'description_width': '150px'} 
box_layout = widgets.Layout(display='flex',
                flex_flow='column',
                align_items='center',
                width='100%')

# Admin Form
cb_correctAnswerId = NULL
tb_question= NULL
tb_answer1 = NULL
tb_answer2 = NULL
tb_answer3 = NULL
tb_answer4 = NULL
offset = 0

########## Functions
def get_all_database():
  global questionsFromDB
  FBconn = firebase.FirebaseApplication(FIREBASE_PATH,None)
  res = FBconn.get('/Questions/',None)

  if res:
    # insert questions to DB class
    question_db = QuestionsDB([question for question in res])


# Define a function to save the selected answer
def get_question_and_answers_dictionary():
        global randomQuestionAndAnswer, answers, question, radioButtonLst

        randomQuestionAndAnswer = NULL # we want to generate new item...
        if (questionsFromDB):
          while randomQuestionAndAnswer == NULL:
            randomQuestionAndAnswer = random.choice(questionsFromDB)
            if str(randomQuestionAndAnswer['question']) in questionsHistory:
              randomQuestionAndAnswer = NULL
        else:
          return

        if randomQuestionAndAnswer != NULL:            
          question = str(randomQuestionAndAnswer['question'])
          questionsHistory.add(question)
          answers = [str(ans) for ans in randomQuestionAndAnswer['answers']]
          answers = answers[1:]

def show_statistics():
  """
  Generate and show statistics plot. 
  The statistics are number of attempts for each question
  """
  global list_of_stats
  list_of_stats.insert(0, 0) # normalize the data
  fig, ax = plt.subplots() # configure
  
  ax.plot(list_of_stats) # make the line plot

  # mark the dots
  for i, stat in enumerate(list_of_stats):
    if stat != 0:
        ax.scatter(i, stat, color='green', marker='o')

  # make data readable (start from 1)
  ax.set_xticks(range(1, len(list_of_stats))) 
  ax.set_yticks(range(1, 5))

  ax.set_title("Your game statistics")
  ax.set_ylabel("Number of attempts")
  ax.set_xlabel("Question #")
  
  plt.show()
  

def game_over():

  """
  End game - show the score and statistics
  """
  output.clear()
  good_words = ""
  badge_src = ""
  total_score_to_earn = 3*len(questionsFromDB)
  name = current_user.getName()
  if total_score_to_earn < 2:
    good_words = f"{name}, You are AWESOME!"
    badge_src = "https://thumbs.dreamstime.com/z/gold-medal-st-place-badge-symbol-victory-sport-business-gold-medal-st-place-badge-symbol-victory-sport-business-128259723.jpg"
  elif total_score_to_earn - points < 15:
    good_words = f"{name}, You are great! Try again to beat yourself "
    badge_src = "https://c8.alamy.com/comp/JE56PM/silver-2st-place-medal-vector-metal-realistic-badge-with-second-placement-JE56PM.jpg"
  else:
    good_words = f"{name}, You are good, but need more practice. try out our courses and play again"
    badge_src = "https://thumbs.dreamstime.com/b/award-badge-third-place-red-ribbon-isolated-medal-pin-symbol-realistic-d-design-victory-trophy-sign-number-white-147089746.jpg"
  succ_percentages = round((points/total_score_to_earn)*100)
  

  display(IPython.display.HTML(f"<div style='text-align: center; justify-content: center; align-items: center;'>"+
  f"<h2 style='color: red'><img src='https://www.animatedimages.org/data/media/492/animated-fireworks-image-0087.gif' style='width: 50px; height: 50px'/>"+
                               f"Game over!<img src='https://www.animatedimages.org/data/media/492/animated-fireworks-image-0087.gif' style='width: 50px; height: 50px'/></h2>"+
  f"<h4><u>Your score</u></h4><br/><strong>Points:</strong> {points} | "+
  f"<strong>Correct answers:</strong> {quantity_of_correct_questions}"
  f"<br/><strong>You could earn up to</strong> {total_score_to_earn} points."+
  f"<br/><strong>Your success rate is:</strong> {succ_percentages}%"+
  
  f"<h4 style='color: #00bfff;'>{good_words}</h4><br/>"+
  f"<div>You earn a new badge: <img src={badge_src} style='width: 150px; height: 150px; display:block; margin:auto; '/></div></div>"))
  show_statistics()


  
def save_answer(b):
  """
  On click on some answer
  """
  global currentSelectedAns, finalSelectedBtn
  currentSelectedAns = str(b.description)  # save the answer to compare
  finalSelectedBtn = b  # save the button object to mark it later
  on_answer_button_clicked(b)
  update_total_questions_quantity()
  if 3 == len(questionsHistory) - 1: # LIDOR - CHANGE this to len(questionsFromDB) instead of 3
      game_over()
      return

def create_and_show_elements():
  """
  Create questions page elemnts and show them
  """
  global randomQuestionAndAnswer, answers, question, radioButtonLst
  display(IPython.display.HTML(f"<h2 style='display: flex; justify-content: center; align-items: center;'>{question}</h2>"))
  points_label = HTML(f"<strong>Points:</strong> {points}")
  correct_label = HTML(f"<strong>Correct Questions:</strong> {quantity_of_correct_questions}")
  total_label = HTML(f"<strong>Out Of:</strong> {quantity_of_questions}")

  box = widgets.HBox(children=[points_label, correct_label, total_label],layout=box_layout)
  display(box)
  
  buttonsLst = [widgets.Button(description = answer, layout = widgets.Layout( width='auto', height='auto'),
                    style={'white_space': 'pre-wrap'}
                  ) for answer in answers]

  for btn in buttonsLst:
    btn.on_click(save_answer)
    btn.style.button_color = 'orange'

  left_box = VBox([buttonsLst[0], buttonsLst[1]])
  right_box = VBox([buttonsLst[2], buttonsLst[3]])
  
  box = widgets.HBox(children=[HBox([left_box, right_box])],layout=box_layout)
  display(box)

def generate_new_question():
  """
  Generate a new question
  """
  global show_go_back
  output.clear()
  get_question_and_answers_dictionary()
  create_and_show_elements()
  btn_go_back.description = "Exit game"
  show_go_back(create_menu_buttons)  

def on_answer_button_clicked(b):
    """
    The true logic for answer button clicked
    """
    global finalSelectedBtn, list_of_stats
    # iterate over question widgets and get user answers
    answerId = randomQuestionAndAnswer['correctAnswer']
    if answerId:
      correctAns = answers[int(answerId)-1]

    if currentSelectedAns is NULL:
      print("Please select answer")
      return
    list_of_stats[len(questionsHistory)-1] += 1

    # check if user's answer is correct
    if currentSelectedAns.strip() == correctAns.strip():
        update_points(3)
        update_total_correct_questions_quantity()
        print("Correct!")
        finalSelectedBtn.style.button_color = 'lightgreen'

        # start timer and refresh
        time.sleep(1.5)
        generate_new_question()
    else:
        update_points(-1)
        finalSelectedBtn.style.button_color = 'red'
        display(IPython.display.HTML('<br/><h4>Incorrect answer<br/>Try Again! You can do it!<h4/><br/>'))

def update_points(new_points):
    """
    Update and display current points
    """
    global points
    points += new_points
    display(IPython.display.Markdown(f"**New Points:** {points}"))
    correct_label = IPython.display.Markdown(f"**Correct Questions:** {quantity_of_correct_questions}")
    total_label = IPython.display.Markdown(f"**Out Of:** {quantity_of_questions}")
    display(correct_label)
    display(total_label)

def update_total_questions_quantity():
    global quantity_of_questions
    quantity_of_questions+=1

def update_total_correct_questions_quantity():
    global quantity_of_correct_questions
    quantity_of_correct_questions+=1

def insert_new_question_to_db(b):
  """
  Inserting new question. Return true if succeeded
  """
  global offset  
  # Connect to DB
  try:
    # TODO: check validation (not empty, no repeats)
    data_to_upload = {
      'question' : tb_question.value,
      'answers' : [None, tb_answer1.value, tb_answer2.value, tb_answer3.value, tb_answer4.value],
      'correctAnswer' : cb_correctAnswerId.value
    }
    FBconn = firebase.FirebaseApplication(FIREBASE_PATH,None)
    result = FBconn.put("/Questions", name=f"{len(questionsFromDB)+offset+1}", data=data_to_upload)
    offset += 1
    print("Success")
    return True
  except Exception as e:
    print(f"Can't insert into DB\n{e}")
    return False


def show_form():
    """
    Present a form for adding new questions into DB.
    """
    global cb_correctAnswerId, tb_question, tb_answer1, tb_answer2, tb_answer3, tb_answer4, offset
    output.clear()
    cb_correctAnswerId = widgets.Dropdown(options=['1','2','3','4'],  description='Correct answer Number:',
        disabled=False, style=style)

    tb_question = widgets.Text(description='Question:', style=style,)
    tb_answer1 = widgets.Text(description='Answer #1:', style=style,)
    tb_answer2 = widgets.Text(description='Answer #2:', style=style,)
    tb_answer3 = widgets.Text(description='Answer #3:', style=style,)
    tb_answer4 = widgets.Text(description='Answer #4:', style=style,)
    
    btn_insert = widgets.Button(description='Insert', disabled=False, 
                            button_style='success', 
                            layout=Layout(margin="20px 0px 0px 140px"))
    btn_insert.on_click(insert_new_question_to_db)

    display(VBox([tb_question, tb_answer1, tb_answer2, tb_answer3, tb_answer4, cb_correctAnswerId, btn_insert]))
    show_go_back(create_menu_buttons)

def show_go_back(back_event):
  if btn_go_back != NULL:
    display(btn_go_back)
  btn_go_back.on_click(back_event)

def on_admin_clicked(b):
    """
    Handle validation of admin
    """
    output.clear()
    tb_password = widgets.Password(description='Password:',  style=style, placeholder='Insert password')
    tb_password.value = ""
    btn_submit_password = widgets.Button(description="Login", layout=Layout(margin="0px 0px 0px 155px"))

    def is_admin_password_valid(b):
      if tb_password.value == "123123":
        show_form()
      else:
        print("Invalid password")

    btn_submit_password.on_click(is_admin_password_valid)
    display(VBox([tb_password, btn_submit_password]))
    display(IPython.display.HTML(f"<br/><br/>"))
    show_go_back(create_menu_buttons)

def create_menu_buttons(b=NULL):
    """
    Present the main menu
    """
    output.clear()
    global btn_go_back, list_of_stats
    btn_questions = widgets.Button(description="Questions")
    btn_classes = widgets.Button(description="Classes")
    btn_leaderboard = widgets.Button(description="Leaderboard")
    btn_help = widgets.Button(description="Help")
    btn_admin = widgets.Button(description="Add Questions (Admin only)", 
                               layout=Layout(width="200px"))
    btn_go_back = widgets.Button(description="Back", button_style="primary",layout=Layout(width="80px"))
    
      
    
    def on_questions_clicked(b):
        global list_of_stats
        output.clear()
        list_of_stats = [0]*len(questionsFromDB)
        generate_new_question()
      

    def on_leaderboard_clicked(b):
        output.clear()
        # 
        display(IPython.display.HTML(f"<div style='text-align: center; justify-content: center; align-items: center;'>"+
        f"<h2 style='color: red'><img src='https://media.tenor.com/8QEKfyK2RUcAAAAM/rotating-star.gif' style='width: 50px; height: 50px'/>"+
                                    f"Leaderboard!<img src='https://media.tenor.com/8QEKfyK2RUcAAAAM/rotating-star.gif' style='width: 50px; height: 50px'/></h2>"+
        f"</h2></div>"))
        display(IPython.display.HTML(f"<h4>Coming up soon</h4>"))
        show_go_back(create_menu_buttons)
        
    def on_classes_clicked(b):
        output.clear()
        display(IPython.display.HTML(f"<h2>Classes page</h2>"))
        display(IPython.display.HTML(f"<h4>To gain access to our exceptional"+
        " classes, kindly get in touch with our customer service team to arrange payment.</h4>"))
        show_go_back(create_menu_buttons)

    
    def on_help_clicked(b):
        output.clear()
        display(IPython.display.HTML(f"<h2>Help page</h2>"))
        display(IPython.display.HTML(f"<h4>Answer as many questions as possible "+ 
          f"with the fewest attempts.<br/>Each correct answer is worth 3 points, "+
          f"but each incorrect answer will cost you 1 point</h4>"))
        display(IPython.display.HTML(f"<br/><br/><h4>Have Fun!</h4>"))
        show_go_back(create_menu_buttons)


    # Assign callbacks to buttons
    btn_questions.on_click(on_questions_clicked)
    btn_classes.on_click(on_classes_clicked)
    btn_help.on_click(on_help_clicked)
    btn_admin.on_click(on_admin_clicked)
    btn_leaderboard.on_click(on_leaderboard_clicked)
    # Display Menu buttons
    if current_user.isAdmin():
      box = widgets.HBox(children=[btn_questions,btn_leaderboard, btn_classes, btn_help, btn_admin], layout=box_layout)
    else:
      box = widgets.HBox(children=[btn_questions,btn_leaderboard, btn_classes, btn_help], layout=box_layout)
    display(box)


# Define callbacks
def on_start_clicked(b):
    output.clear()
    create_menu_buttons()
    

def start_game():
    # Create buttons
    btn_start = widgets.Button(description="Start Now")
    btn_start.on_click(on_start_clicked)
    box = widgets.HBox(children=[btn_start],layout=box_layout)
    display(box)

##################################
############ LOGIN ###############
##################################

def show_h4_center_text(text):
  """
  Generic function for h4 centered text
  """
  display(IPython.display.HTML(f"<div style='text-align: center; justify-content: center; align-items: center;'>"+
    f"<h4>{text}</h4><br/>"))

def show_login_page():
    """
    Create and show the login page elements
    """
    tb_username = widgets.Text(description='Username:', style=style,)
    tb_password = widgets.Password(description='Password:', style=style,)
    
    btn_login = widgets.Button(description='Login', disabled=False, 
                            button_style='success', 
                            layout=Layout(margin="20px 0px 0px 140px"))
    
    def temp(b):
          validate_login(tb_username.value, tb_password.value)
    btn_login.on_click(temp)

    display(VBox([tb_username, tb_password, btn_login]))


def sign_up(username, password):
  """
  Insert new user to DB
  """
  # Connect to DB
  try:
    # TODO: check validation (not empty, no repeats)
    data_to_upload = {
      'Username' : f'{username}',
      'Password' : f'{password}',
      'Role' : 'User'
    }
    FBconn = firebase.FirebaseApplication(FIREBASE_PATH,None)
    result = FBconn.post("/Users", data=data_to_upload)

    display(IPython.display.HTML(f"<div style='text-align: center; justify-content: center; align-items: center;'>"+
    f"<h4>You registered successfully!<br/>Next time you can login with your details</h4>"))
    time.sleep(3)
    create_menu_buttons() # Auto navigate to 

  except Exception as e:
    print(f"Can't insert into DB\n{e}")
    return False

def is_user_Exist(username):
  FBconn = firebase.FirebaseApplication(FIREBASE_PATH,None)
  users = FBconn.get('/Users/', None)
  for user_id, user_data in users.items():
      if user_data['Username'] == username:
          return True 
  return False

  
def validate_login(username, password):
  """
  Validate the login info against DB, and suggest to register if need, or starting the game
  """
  def get_user():
    global current_user
    FBconn = firebase.FirebaseApplication(FIREBASE_PATH,None)
    users = FBconn.get('/Users/', None)
    for user_id, user_data in users.items():
        if user_data['Username'] == username and user_data['Password'] == password:
            current_user = User(username, password, user_data['Role'])
            return True 
    return False
  

  if not username or not password:
      show_h4_center_text("Please insert username & password")
  else:
      if is_user_Exist(username):
        show_h4_center_text("Incorrect password")
        return
  
      res = get_user()
      if res:
        create_menu_buttons() #return True
      else:
        # show message and suggest to register
        show_h4_center_text("The username / password are incorrect")
        show_h4_center_text("If you are new, please register")
          
        btn_register = widgets.Button(description='Register', disabled=False, 
                            button_style='primary', 
                            layout=Layout(margin="20px 0px 0px 140px"))
        def temp(b):
          sign_up(username, password)
        btn_register.on_click(temp)
        display(VBox([btn_register]))



        



##### MAIN #####

get_all_database()
show_login_page()


{'answers': [None, 'A global variable is defined outside of a function while a local variable is defined inside a function.', 'A global variable is immutable while a local variable is mutable.', 'A global variable is accessible only within a function while a local variable is accessible throughout the program.', 'There is no difference between a global variable and a local variable in Python.'], 'correctAnswer': '1', 'question': 'What is the difference between a global variable and a local variable in Python?'}
{'answers': [None, 'orange', 'red', 'blue', '=]'], 'correctAnswer': '4', 'question': 'pick orange'}
{'answers': [None, 'It returns the length of a string or a list.', 'It returns the current date and time.', 'It returns the square root of a number.', 'It converts a string to a list.'], 'correctAnswer': '1', 'question': 'What does the built-in function len() do in Python?'}
{'answers': [None, 'A module is a collection of related functions while a package is a collection of relate